# I will use a late submission token for this assignment

# Q4

First, we generate length 1 frequent itemsets from local DB. The number of frequent itemsets generated in each DB should be in proportion to the number of transactions in that DB. After the generation, local DBs then submit their length 1 frequent itemsets and corresponding support to the central server.

Second, the central server aggregates all submitted local supports by each itemsets. If the total supports exceed the threshold , which is 0.1 million (1% of the total transactions), that itemset is frequet.

Third, the central server would pass the frequent itemsets generated in central server to local servers. Then the local servers would use those frequent itemsets from the central server to generate new supersets and then pass those supersets to the central server.

Finally, repeat steps above and stop the process until no frequent itemsets are generated in the central server.

# Q5-1

I use mlxtend to meet the question requirement. My methodology is after generated all frequent itemsets, filtered those don't have the "Outcome" attribute so that all frequent pattern will only contain "Outcome".


In [83]:
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd

#1. discretize the data
df = pd.read_csv("diabetes.csv")
cols = ["Pregnancies", "Glucose", "BloodPressure", "Age", 'Outcome']
df = df[cols]
df['Glucose'] = pd.cut(df['Glucose'], bins=range(0, 200, 10), right=False, labels=[f"Glucose_{i}_{i+9}" for i in range(0, 190, 10)])
df['BloodPressure'] = pd.cut(df['BloodPressure'], bins=range(0, 200, 10), right=False, labels=[f"BP_{i}_{i+9}" for i in range(0, 190, 10)])
df['Age'] = pd.cut(df['Age'], bins=range(0, 100, 10), right=False, labels=[f"Age_{i}_{i+9}" for i in range(0, 90, 10)])
df['Outcome'] = df['Outcome'].apply(lambda x: 'Diabetes' if x == 1 else 'No_Diabetes')

In [84]:
#2. Convert all columns to transactions
df = df.astype(str)
transactions = df.values.tolist()

# 3. Apply one-hot-encoding to the dataset
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

# 4. Apply the apriori algorithm
frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)

# 5. Filter frequent itemsets to include those contain 'Outcome' attribute
filtered_frequent_itemsets = frequent_itemsets.copy()
filtered_frequent_itemsets['contains_outcome'] = filtered_frequent_itemsets['itemsets'].apply(lambda x: 'Diabetes' in x or 'No_Diabetes' in x)
filtered_frequent_itemsets = filtered_frequent_itemsets[filtered_frequent_itemsets['contains_outcome']]

# 6. Find top 100 most frequent itemsets.
top_100_itemsets = filtered_frequent_itemsets.sort_values(by='support', ascending=False).head(100)

# Q5-2

In [85]:
for index, row in top_100_itemsets.iterrows():
    print(f"Row {index}: {row.to_dict()}")

Row 40: {'support': 0.6510416666666666, 'itemsets': frozenset({'No_Diabetes'}), 'contains_outcome': True}
Row 167: {'support': 0.40625, 'itemsets': frozenset({'No_Diabetes', 'Age_20_29'}), 'contains_outcome': True}
Row 27: {'support': 0.3489583333333333, 'itemsets': frozenset({'Diabetes'}), 'contains_outcome': True}
Row 237: {'support': 0.19921875, 'itemsets': frozenset({'No_Diabetes', 'BP_70_79'}), 'contains_outcome': True}
Row 225: {'support': 0.18619791666666666, 'itemsets': frozenset({'No_Diabetes', 'BP_60_69'}), 'contains_outcome': True}
Row 403: {'support': 0.140625, 'itemsets': frozenset({'No_Diabetes', 'BP_60_69', 'Age_20_29'}), 'contains_outcome': True}
Row 71: {'support': 0.13802083333333334, 'itemsets': frozenset({'No_Diabetes', '1'}), 'contains_outcome': True}
Row 301: {'support': 0.12239583333333333, 'itemsets': frozenset({'No_Diabetes', 'Age_20_29', '1'}), 'contains_outcome': True}
Row 226: {'support': 0.11979166666666667, 'itemsets': frozenset({'BP_70_79', 'Diabetes'}), 

# Q5-3

In [86]:


# 7. Display the rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.01)

# 8. filter out those consequents is not the outcome
filtered_rules = rules[rules['consequents'].apply(lambda x: 'Diabetes' in x or 'No_Diabetes' in x)]

filtered_rules_sorted = filtered_rules.sort_values(by='confidence', ascending=False) 
filtered_rules_sorted.head(5)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
2097,"(Age_20_29, Glucose_70_79, 1)",(No_Diabetes),0.010417,0.651042,0.010417,1.0,1.536,0.003635,inf,0.352632
2363,"(Glucose_100_109, BP_50_59, Age_20_29)",(No_Diabetes),0.016927,0.651042,0.016927,1.0,1.536,0.005907,inf,0.354967
850,"(2, Glucose_80_89)",(No_Diabetes),0.016927,0.651042,0.016927,1.0,1.536,0.005907,inf,0.354967
2251,"(2, Glucose_120_129, Age_20_29)",(No_Diabetes),0.016927,0.651042,0.016927,1.0,1.536,0.005907,inf,0.354967
2139,"(Glucose_80_89, BP_60_69, 1)",(No_Diabetes),0.011719,0.651042,0.011719,1.0,1.536,0.004089,inf,0.353096
